In [ ]:
# Installing all necessary libraries
#!pip3 install xgboost
import pandas as pd
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# load dataset
df=pd.read_csv("PipesFinal.csv")

In [ ]:
df=df.fillna(0)

In [ ]:
# One-hot encode the Province feature
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
province_onehot = onehot_encoder.fit_transform(df[["Province"]]).toarray()
province_onehot_df = pd.DataFrame(province_onehot, columns=onehot_encoder.get_feature_names(["Province"]))

# Drop the original Province feature and concatenate the one-hot encoded features
df = df.drop("Province", axis=1)
df = pd.concat([df, province_onehot_df], axis=1)

C:\Users\rpola\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# One-hot encode the Material feature
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
material_onehot = onehot_encoder.fit_transform(df[["Material"]]).toarray()
material_onehot_df = pd.DataFrame(material_onehot, columns=onehot_encoder.get_feature_names(["Material"]))

# Drop the original Material feature and concatenate the one-hot encoded features
df = df.drop("Material", axis=1)
df = pd.concat([df, material_onehot_df], axis=1)

C:\Users\rpola\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# One-hot encode the GasType feature
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
gastype_onehot = onehot_encoder.fit_transform(df[["GasType"]]).toarray()
gastype_onehot_df = pd.DataFrame(gastype_onehot, columns=onehot_encoder.get_feature_names(["GasType"]))

# Drop the original GasType feature and concatenate the one-hot encoded features
df = df.drop("GasType", axis=1)
df = pd.concat([df, gastype_onehot_df], axis=1)

C:\Users\rpola\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Split the data into features and target variables
X = df.drop(["Unnamed: 0","MaintenanceId","Incidence","Town","MaintenanceId","Severity","BoolBridle",'Month','Day','Year'], axis=1)
y = df["Incidence"]

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=0)

In [1]:
# X_train.head()

In [ ]:
|from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
model = xgb.XGBClassifier(learning_rate=0.015, 
                          n_estimators=170,
                          max_depth=20,
                          reg_lambda=1,
                          colsample_bytree=0.9,
                          gamma=0,
                          eval_metric="auc",
                          objective='binary:logistic')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=170, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
# make predictions for test data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print("AUC score:", auc_score)

AUC score: 0.9088934336361503


In [ ]:
sample_sub=pd.read_csv("sample_submission.csv")

In [ ]:
dftest=df.drop_duplicates(subset="PipeId",keep="first")

In [ ]:
dftest1=pd.merge(sample_sub,dftest, on="PipeId", how="left")

In [ ]:
dftest1=dftest1.drop(["Incidence_x","Unnamed: 0","MaintenanceId","Incidence_y","Town","BoolBridle","Severity","Month","Day","Year"],axis=1)

In [ ]:
sample_proba = model.predict_proba(dftest1)[:, 1]

In [ ]:
sample_proba

array([0.0391043 , 0.03957758, 0.0391043 , ..., 0.03923157, 0.03916085,
       0.03910968], dtype=float32)

In [ ]:
dftest1['Incidence'] = sample_proba.tolist()

In [ ]:
dftest1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 909733 entries, 0 to 909732
Data columns (total 64 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   PipeId                909733 non-null  int64  
 1   MonthsLastRev         909733 non-null  float64
 2   YearBuilt             909733 non-null  int64  
 3   Diameter              909733 non-null  float64
 4   Length                909733 non-null  float64
 5   Pressure              909733 non-null  float64
 6   NumConnections        909733 non-null  int64  
 7   NumConnectionsUnder   909733 non-null  int64  
 8   BoolBridleFinal       909733 non-null  int64  
 9   flow_rate             909733 non-null  float64
 10  TotalInspections      909733 non-null  int64  
 11  InspectionYear        909733 non-null  float64
 12  Province_Albacete     909733 non-null  float64
 13  Province_Alicante     909733 non-null  float64
 14  Province_Almeria      909733 non-null  float64
 15  

In [ ]:
submission=dftest1.drop(dftest1.iloc[:, 1:63],axis=1)

In [ ]:
submission.head()

,PipeId,Incidence
0,446859944,0.039104
1,428124500,0.039578
2,438428871,0.039104
3,429034569,0.039689
4,411184477,0.039382


In [ ]:
submission.isna().sum()

PipeId       0
Incidence    0
dtype: int64

In [ ]:
submission.to_csv("Submission29.csv")